In [18]:
#Notebook which provides best troughput achievable for a given M, N and d with existence of each edge being variable
#I'll probably make this into a proper python file soon, but it's more convenient this way for now
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import numpy as np

In [19]:
N = 16
dE = 8

In [20]:
#Specify the demand matrix
# demand = np.zeros((N,N))
# for i in range(N):
#     for j in range(N):
#         if i!=j:
#             demand[i][j] = dE/(N-1)


workdir="/home/studium/Documents/Code/rdcn-throughput/matrices/"
demand= np.loadtxt(workdir+"skew-16-0.1.mat", usecols=range(N))
demand = demand * dE

In [21]:
#Initialize the model
model = gp.Model("throughput")
#TODO: Learn more about tolerances and which to use
# model.Params.OptimalityTol = 1e-9
# model.Params.FeasibilityTol = 1e-9


In [22]:
#Add edge vars
edge_vars = {}
for i in range(N):
    for j in range(N):
        edge_vars[i, j] = model.addVar(vtype= GRB.INTEGER, name=f"edge_{i}_{j}",ub=dE)

In [23]:
#Inc edges constraint
for i in range(N):
    model.addConstr(gp.quicksum(edge_vars[i,j] for j in range(N))<=dE)

In [24]:
#Outg edges constraint
for i in range(N):
    model.addConstr(gp.quicksum(edge_vars[j,i] for j in range(N))<=dE)

In [25]:
#Add flow vars for every node combination
flow_vars = {}
for s in range(N):
    for d in range(N):
        if demand[s,d] > 0:
            for i in range(N):
                for j in range(N):
                    flow_vars[s, d, i, j] = model.addVar(vtype= GRB.CONTINUOUS, name=f"flow_{s}_{d}_{i}_{j}",lb=0)

In [26]:
#Add throughput variable
throughput = model.addVar(vtype=GRB.CONTINUOUS,name='throughput',lb=0,ub=1)

In [27]:
#Source demand constraint
for s in range(N):
    for d in range(N):
            if demand[s,d] > 0:
                model.addConstr(gp.quicksum(flow_vars[s, d, s, i] for i in range(N) if i!=s )>= demand[s,d]* throughput,name =f"sdconst_{s}_{d}")

In [28]:
#Dest demand constraint
for s in range(N):
    for d in range(N):
            if demand[s,d] > 0:
                model.addConstr(gp.quicksum(flow_vars[s, d, i, d] for i in range(N) if i!=d )>= demand[s,d]* throughput,name =f"ddconst_{s}_{d}")

In [29]:
#Flow conservation
for s in range(N):
    for d in range(N):
        if demand[s,d] > 0:
            for j in (j for j in range(N) if j != s and j != d):
                model.addConstr(gp.quicksum(flow_vars[s, d, i, j] for i in range(N) if i!=j )-gp.quicksum(flow_vars[s, d, j, k] for k in range(N) if k!=j)== 0,name =f"fcconst_{s}_{d}_{j}")

In [30]:
#Capacity Constraints
for i in range(N):
    for j in range(N):
        model.addConstr(gp.quicksum(flow_vars[s,d,i,j] for d in range(N) for s in range(N) if demand[s,d] > 0)<= edge_vars[i,j],name =f"cconst_{s}_{d}")

In [31]:
#Set the objective: maximize throughput
model.setObjective(throughput, GRB.MAXIMIZE)

In [32]:
model.optimize()
print("Throughput for the given topology and the given demand matrix is:", throughput.X)
for v in model.getVars():
        if v.x != 0:
                print(v.varName, "=", v.x)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "TUXEDO OS 2")

CPU model: AMD Ryzen 5 5500U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 288 rows, 257 columns and 768 nonzeros
Model fingerprint: 0xe42f49bd
Variable types: 1 continuous, 256 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 8e+00]
  RHS range        [8e+00, 8e+00]
Found heuristic solution: objective 1.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+00, best bound 1.000000000000e+00, gap 0.0000%
Throughput for the given topology and the given demand matrix is: 1.0
throughput = 1.0
